In [2]:
#here is an example of how to build and populate a hurestic model

import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/daniel-dc-cd/data_science/master/module_4_ML/data/seattle_weather_1948-2017.csv')

numrows = 25549 # can be as large as 25549

#create an empty dataframe to hold 100 values
heuristic_df = pd.DataFrame({'yesterday':[0.0]*numrows,
                             'today':[0.0]*numrows,
                             'tomorrow':[0.0]*numrows,
                             'guess':[False]*numrows, #logical guess
                             'rain_tomorrow':[False]*numrows, #historical observation
                             'correct':[False]*numrows, #TRUE if your guess matches the historical observation
                             'true_positive':[False]*numrows, #TRUE If you said it would rain and it did
                             'false_positive':[False]*numrows,#TRUE If you sait id would rain and it didn't
                             'true_negative':[False]*numrows, #TRUE if you said it wouldn't rain and it didn't
                             'false_negative':[False]*numrows}) #TRUE if you said it wouldn't raing and it did

#sort columns for convience
seq = ['yesterday',
       'today',
       'tomorrow',
       'guess',
       'rain_tomorrow',
       'correct',
       'true_positive',
       'false_positive',
       'true_negative',
       'false_negative']
heuristic_df = heuristic_df.reindex(columns=seq)

In [3]:
df.head()

,DATE,PRCP,TMAX,TMIN,RAIN
0,1948-01-01,0.47,51,42,True
1,1948-01-02,0.59,45,36,True
2,1948-01-03,0.42,45,35,True
3,1948-01-04,0.31,45,34,True
4,1948-01-05,0.17,45,32,True


In [4]:
heuristic_df.head()

,yesterday,today,tomorrow,guess,rain_tomorrow,correct,true_positive,false_positive,true_negative,false_negative
0,0.0,0.0,0.0,False,False,False,False,False,False,False
1,0.0,0.0,0.0,False,False,False,False,False,False,False
2,0.0,0.0,0.0,False,False,False,False,False,False,False
3,0.0,0.0,0.0,False,False,False,False,False,False,False
4,0.0,0.0,0.0,False,False,False,False,False,False,False


In [6]:
# here is an example loop that populates the dataframe created earlier
# with the total percip from yesterday and today
# then the guess is set to true if rained both yesterday and today 
for z in range(numrows):
    #start at time 2 in the data frame
    i = z + 2
    #pull values from the dataframe
    yesterday = df.iloc[(i-2),1]
    today = df.iloc[(i-1),1]
    tomorrow = df.iloc[i,1]
    rain_tomorrow = df.iloc[(i),1]
    
    heuristic_df.iat[z,0] = yesterday
    heuristic_df.iat[z,1] = today
    heuristic_df.iat[z,2] = tomorrow
    heuristic_df.iat[z,3] = False # set guess default to False
    heuristic_df.iat[z,4] = rain_tomorrow
    
    #example hueristic
    if today > 0.0 and yesterday > 0.0:
        heuristic_df.iat[z,3] = True
    if heuristic_df.iat[z,3] == heuristic_df.iat[z,4]:
        heuristic_df.iat[z,5] = True
        if heuristic_df.iat[z,3] == True:
            heuristic_df.iat[z,6] = True #true positive
        else:
            heuristic_df.iat[z,8] = True #true negative
    else:
        heuristic_df.iat[z,5] = False
        if heuristic_df.iat[z,3] == True:
            heuristic_df.iat[z,7] = True #false positive
        else:
            heuristic_df.iat[z,9] = True #false negative

In [11]:
from sklearn.model_selection import train_test_split 
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# enter split function here to make h_train and h_test subsets of the data
h_train , h_test=train_test_split(heuristic_df , test_size=.8)
h_train.head()

,yesterday,today,tomorrow,guess,rain_tomorrow,correct,true_positive,false_positive,true_negative,false_negative
3660,0.22,0.32,0.54,True,True,True,True,False,False,False
19392,0.17,0.37,0.00,True,False,False,False,True,False,False
18662,0.04,0.12,0.48,True,True,True,True,False,False,False
22837,0.00,0.00,0.00,False,False,True,False,False,True,False
25470,0.01,0.00,0.00,False,False,True,False,False,True,False


In [14]:
h_test.head()

,yesterday,today,tomorrow,guess,rain_tomorrow,correct,true_positive,false_positive,true_negative,false_negative
3660,0.22,0.32,0.54,True,True,True,True,False,False,False
19392,0.17,0.37,0.00,True,False,False,False,True,False,False
18662,0.04,0.12,0.48,True,True,True,True,False,False,False
22837,0.00,0.00,0.00,False,False,True,False,False,True,False
25470,0.01,0.00,0.00,False,False,True,False,False,True,False


In [22]:
# we used this simple approach in the first part to see what percent of the time we where correct 
# calculated as (true positive + true negative)/ number of guesses
num_of_guesses=0

for i in heuristic_df['guess']:
    num_of_guesses=num_of_guesses+1

num_of_guesses

25549

In [23]:
truepositive=0

for i in heuristic_df['true_positive']:
    truepositive=truepositive+i  

truepositive

4861

In [24]:
negative=0

for i in heuristic_df['true_negative']:
    negative=negative+i 
 

negative

12298

In [26]:
accuracy = (truepositive + negative)/num_of_guesses
accuracy

0.6716114133625583

In [33]:
# precision is the percent of your postive prediction which are correct
# more specifically it is calculated (num true positive)/(num tru positive + num false positive)
num_of_TRUEPOSITIVE=0
for i in heuristic_df['true_positive']:
    num_of_TRUEPOSITIVE=num_of_TRUEPOSITIVE+1
num_of_TRUEPOSITIVE 

25549

In [32]:
num_false_positive = 0
for i in heuristic_df['false_positive']:
    num_false_positive=num_false_positive+1
num_false_positive

25549

In [31]:
precision_prediction= num_of_TRUEPOSITIVE/(num_of_TRUEPOSITIVE+num_false_positive)
precision_prediction

0.5

In [37]:
# recall the percent of the time you are correct when you predict positive
# more specifically it is calculated (num true positive)/(num tru positive + num false negative)

num_false_negative = 0

for i in heuristic_df['false_negative']:
    num_false_negative=num_false_negative+1
num_false_negative

25549

In [48]:
recall_of_predictions = num_of_TRUEPOSITIVE/(num_of_TRUEPOSITIVE+num_false_negative)
recall_of_predictions

0.5

In [47]:
#The sum of squared error (SSE) of your predictions
 #looping through each element of the list
summation=0
difference = precision_prediction -  recall_of_predictions  #finding the difference between observed and predicted value
squared_difference = difference**2  #taking square of the differene 
  
summation = summation + squared_difference
summation

0.0